<a href="https://colab.research.google.com/github/rlberry-py/tutorials/blob/main/A2C/Tutorial_Advantage_Actor_Critic_(A2C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial - Advantage Actor Critic (A2C)

A2C keeps two neural networks:
*   One network with paramemeters $\theta$ to represent the policy $\pi_\theta$.
*   One network with parameters $\omega$ to represent a value function $V_\omega$, that approximates $V^{\pi_\theta}$


At each iteration, A2C collects $M$ transitions $(s_i, a_i, r_i, s_i')_{i=1}^M$ by following the policy $\pi_\theta$. If a terminal state is reached, we simply go back to the initial state and continue to play $\pi_\theta$ until we gather the $M$ transitions.

Consider the following quantities, defined based on the collected transitions:

$$
\widehat{V}(s_i) = \widehat{Q}(s_i, a_i) = \sum_{t=i}^{\tau_i \wedge M} \gamma^{t-i} r_t + \gamma^{M-i+1} V_\omega(s_M')\mathbb{I}\{\tau_i>M\}
$$

where and $\tau_i = \min\{t\geq i: s_i' \text{ is a terminal state}\}$, and 

$$
\mathbf{A}_\omega(s_i, a_i) = \widehat{Q}(s_i, a_i) -  V_\omega(s_i)  
$$


A2C then takes a gradient step to minimize the policy "loss" (keeping $\omega$ fixed):

$$
L_\pi(\theta) =
-\frac{1}{M} \sum_{i=1}^M \mathbf{A}_\omega(s_i, a_i) \log \pi_\theta(a_i|s_i)
- \frac{\alpha}{M}\sum_{i=1}^M \sum_a  \pi(a|s_i) \log \frac{1}{\pi(a|s_i)}
$$

and a gradient step to minimize the value loss (keeping $\theta$ fixed):

$$
L_v(\omega) = \frac{1}{M} \sum_{i=1}^M \left( \widehat{V}(s_i) - V_\omega(s_i)   \right)^2
$$
 


# Reminders


Objective function:

$$
J(\theta) = \mathbb{E}_{\pi_\theta}
\left[ 
  \sum_{t=0}^\infty \gamma^t r(S_t, A_t)
\right]
$$

Policy gradient:

$$
\nabla_\theta J(\theta)= \mathbb{E}_{\pi_\theta}
\left[ 
  \sum_{t=0}^\infty \gamma^t A^{\pi_\theta}(S_t, A_t) 
  \nabla_\theta \log \pi_\theta(A_t|S_t)
\right]
$$
where $A^{\pi_\theta}(s, a) = Q^{\pi_\theta}(s, a) - V^{\pi_\theta}(s) $ is the advantage function.

# Colab setup

In [1]:
# After installing, restart the kernel

# install rlberry library
!git clone https://github.com/rlberry-py/rlberry.git 
!cd rlberry && git pull && pip install -e .[full] > /dev/null 2>&1
!pip install ffmpeg-python > /dev/null 2>&1

# gym
!pip install 'gym[all]' > /dev/null 2>&1

# packages required to show video
!pip install pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# ask to restart runtime
print("")
print(" ~~~  Libraries installed, please restart the runtime! ~~~ ")
print("")

Cloning into 'rlberry'...
remote: Enumerating objects: 472, done.
remote: Counting objects: 100% (472/472), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 3541 (delta 283), reused 326 (delta 177), pack-reused 3069
Receiving objects: 100% (3541/3541), 886.51 KiB | 9.85 MiB/s, done.
Resolving deltas: 100% (2277/2277), done.
Already up to date.

 ~~~  Libraries installed, please restart the runtime! ~~~ 



In [7]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40)  # error only

import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch import optim

import numpy as np


# for videos
import rlberry.colab_utils.display_setup
from rlberry.colab_utils.display_setup import show_video

In [8]:
class ActorNetwork(nn.Module):
    """
     This network represents the policy
    """

    def __init__(self, input_size, hidden_size, action_size):
        super(ActorNetwork, self).__init__()
        self.n_actions = action_size
        self.dim_observation = input_size
        
        self.net = nn.Sequential(
            nn.Linear(in_features=self.dim_observation, out_features=hidden_size),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=hidden_size),
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=self.n_actions),
            nn.Softmax(dim=-1)
        )
        
    def policy(self, state):
        state = torch.tensor(state, dtype=torch.float)
        return self.net(state)
    
    def sample_action(self, state):
        state = torch.tensor(state, dtype=torch.float)
        action = torch.multinomial(self.policy(state), 1)
        return action.item()

In [9]:
class ValueNetwork(nn.Module):
  """
   This class represents the value function
  """

  def __init__(self, input_size, hidden_size, output_size):
      super(ValueNetwork, self).__init__()
      self.fc1 = nn.Linear(input_size, hidden_size)
      self.fc2 = nn.Linear(hidden_size, hidden_size)
      self.fc3 = nn.Linear(hidden_size, output_size)

  def forward(self, x):
      out = F.relu(self.fc1(x))
      out = F.relu(self.fc2(out))
      out = self.fc3(out)
      return out
  
  def value(self, state):
      state = torch.tensor(state, dtype=torch.float)
      return self.forward(state)

In [10]:
# You can select your environment here
env_id = 'CartPole-v1'  # @param ["CartPole-v1", "LunarLander-v2", "MountainCar-v0"]
env = gym.make(env_id)
eval_env = gym.make(env_id) # environment to evaluate the policy

In [ ]:
# Define you networks
value_network = ValueNetwork(env.observation_space.shape[0], 16, 1)
actor_network = ActorNetwork(env.observation_space.shape[0], 16, env.action_space.n)
print(value_network)
print(actor_network)

# Define your optimizers
value_network_optimizer = torch.optim.RMSprop(value_network.parameters(), lr=0.01)
actor_network_optimizer = torch.optim.RMSprop(actor_network.parameters(), lr=0.01)

# --------------------------------------------------------------
# Parameters
# --------------------------------------------------------------
num_iterations = 300     # Number of iterations
batch_size = 512         # How many samples to collect (value of M)
gamma = 1                # Discount factor
alpha = 0.001            # Entropy term coefficient
reward_threshold = 495   # Stop training when the policy achieves this amound of rewards


# --------------------------------------------------------------
# Train
# --------------------------------------------------------------
for iteration in range(num_iterations):
    # Initialize batch storage
    states = np.empty((batch_size,) + env.observation_space.shape, dtype=np.float)        # shape (batch_size, state_dim)
    rewards = np.empty((batch_size,), dtype=np.float)                                     # shape (batch_size, )                                 
    next_states = np.empty((batch_size,) + env.observation_space.shape, dtype=np.float)   # shape (batch_size, state_dim)
    dones = np.empty((batch_size,), dtype=np.bool)                                        # shape (batch_size, ) 
    proba = torch.empty((batch_size,), dtype=np.float)                                    # shape (batch_size, ), store pi(a_t|s_t)
    next_value = 0                               # 
  
    # Intialize environment
    state = env.reset()

  # Generate batch
    for i in range(batch_size):
        action = actor_network.sample_action(state)
        next_state, reward, done, _ = env.step(action)

        states[i] = # ...
        rewards[i] = # ...
        next_states[i] = # ...
        dones[i] = # ...
        proba[i] = # ...

        state = next_state
        if done:
          state = env.reset()

    if not done:
        next_value = value_network.value(next_states[-1]).detach().numpy()[0]

    # compute returns (without bootstrapping)
    returns = np.zeros((batch_size,), dtype=np.float)
    T = batch_size
    for j in range(T):
        returns[T-j-1] = rewards[T-j-1]
        if j > 0:
            returns[T-j-1] += gamma * returns[T-j] * (1 - dones[T-j])
        else:
            returns[T-j-1] += gamma * next_value

    # compute advantage
    values = value_network.value(states)
    advantages = # ...

    # Compute MSE (Value loss)
    value_network_optimizer.zero_grad()
    loss_value = # ...
    loss_value.backward()
    value_network_optimizer.step()

    # Compute entropy term
    dist = actor_network.policy(states)
    entropy_term = -(dist*dist.log()).sum(-1).mean()

    # Compute policy loss
    actor_network_optimizer.zero_grad()
    loss_policy = # ...
    loss_policy += -alpha * entropy_term
    loss_policy.backward()
    actor_network_optimizer.step()

    if( (iteration+1)%10 == 0 ):
        eval_rewards = np.zeros(5)
        for sim in range(5):
            eval_done = False
            eval_state = eval_env.reset()
            while not eval_done:
                eval_action = actor_network.sample_action(eval_state)
                eval_next_state, eval_reward, eval_done, _ = eval_env.step(eval_action)
                eval_rewards[sim] += eval_reward
                eval_state = eval_next_state
        print("Iteration = {}, loss_value = {:0.3f}, loss_policy = {:0.3f}, rewards = {:0.2f}"
              .format(iteration +1, loss_value.item(), loss_policy.item(), eval_rewards.mean()))
        if (eval_rewards.mean() > reward_threshold):
            break

In [ ]:
env = Monitor(env, "./gym-results", force=True, video_callable=lambda episode: True)
for episode in range(1):
    done = False
    state = env.reset()
    while not done:
        action = actor_network.sample_action(state)
        state, reward, done, info = env.step(action)
env.close()
show_video(directory="./gym-results")

# Test other environments!

Try some other environments available in OpenAI gym ([link](https://gym.openai.com/envs/#classic_control)). Suggestion: use `classic control` or `Box2D` environments.